## Subgrounds Tutorials: Lido Exploration


This code demonstrates how to use the Subgrounds library to interact with the Lido Ethereum subgraph and fetch specific financial data for a particular pool with the pool's ID. The data is then processed, manipulated, and displayed in a human-readable format.

- Import the necessary libraries: The code starts by importing the required Python libraries, including `datetime` from the standard library, and `SyntheticField` and `Subgrounds` from the `subgrounds` library. It also imports `pandas` and `numpy` for handling data as DataFrames and performing numerical operations.

- Initialize Subgrounds: The `Subground`s object is initialized as `sg`. This object serves as the main entry point for interacting with different subgraphs and fetching data using GraphQL queries.

- Load Lido Ethereum subgraph: The Lido Ethereum subgraph is loaded using its API URL. This returns a `Subgraph` object named `lido_pool_stETH`, which is used to interact with the Lido Ethereum subgraph.

- Query pool data: The code constructs a query to fetch the `pools` entity from the Lido Ethereum subgraph. It specifies ordering options to sort the data by `createdTimestamp` in descending order and limits the number of results to 30. The `where` clause filters the results to only include the pool with the specified ID.

- Create a SyntheticField for human-readable timestamps: A `SyntheticField` named `datetime` is created for the `Pool` entity. This field takes the original `createdTimestamp` field, converts it to a human-readable format using the `datetime.fromtimestamp()` function, and stores it as a float. This makes it easier to read and understand the timestamp data.

- Create a SyntheticField for decimal conversion: A `SyntheticField` named `inputTokenBalance_decimalConv` is created for the `Pool` entity. This field takes the original `inputTokenBalances` field, divides it by 10^18 to convert it to a decimal representation, and stores it as a float.

- Fetch data and store it in a DataFrame: The `sg.query_df()` function is used to execute the query and fetch the data. It takes a list of the fields to be fetched (created timestamp, cumulative protocol-side revenue, cumulative supply-side revenue, cumulative total revenue, input token balance, output token supply, and total value locked) and returns a pandas DataFrame containing the data.

- Squeeze the DataFrame: The `pool_summary` DataFrame is squeezed using the `squeeze()` function, which turns it into a pandas Series named `squeezed_pool_summary`.

- Format the data: A custom function named `format_decimal()` is defined to format the numbers in the Series to a specified number of decimal places. The `squeezed_pool_summary` Series is then passed through this function, resulting in a new Series named `decimal_pool_summary` with formatted numbers.

- Print the resulting Series: The `decimal_pool_summary` Series is printed to display the fetched and formatted financial data.

The main focus of this code is to demonstrate how the Subgrounds library can be used to interact with a subgraph (Lido Ethereum subgraph) and fetch specific financial data for a particular pool using queries. This data can then be processed, manipulated, and displayed in a human-readable format using other Python libraries like pandas and numpy.




In [4]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField
from subgrounds import Subgrounds
import pandas as pd
import numpy as np

# Initialize Subgrounds
sg = Subgrounds()

# Load a subgraph using its API URL
lido_pool_stETH = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/messari/lido-ethereum')

# Create a FieldPath object for the required fields in the financialsDailySnapshots entity.
# Specify options to sort the data by timestamp in descending order and limit the number of results to 30.
lido_pool_stETH_pool_summary = lido_pool_stETH.Query.pools(
    orderBy=lido_pool_stETH.Pool.createdTimestamp, 
    orderDirection='desc', 
    first=30,
    where=[
        lido_pool_stETH.Pool.id == "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84"
    ]
)

# Create a SyntheticField on the Swap entity called `timestamp`, which will format the timestamp field into something more human readable
lido_pool_stETH.Pool.datetime = SyntheticField(
  lambda createdTimestamp: str(datetime.fromtimestamp(createdTimestamp)),
  SyntheticField.FLOAT,
  lido_pool_stETH.Pool.createdTimestamp
)

# Create a SyntheticField on the Swap entity called `timestamp`, which will format the timestamp field into something more human readable
lido_pool_stETH.Pool.inputTokenBalance_decimalConv = SyntheticField(
  lambda x: x[0] / (10**18) if type(x) is list else x / (10**18),
  SyntheticField.FLOAT,
  lido_pool_stETH.Pool.inputTokenBalances
)

lido_pool_stETH.Pool.outputTokenSupply_decimalConv = SyntheticField(
  lambda x: x[0] / (10**18) if type(x) is list else x / (10**18),
  SyntheticField.FLOAT,
  lido_pool_stETH.Pool.outputTokenSupply
)

# lido_pool_stETH.Pool.inputTokenBalance_decimalConv = lido_pool_stETH.Pool.inputTokenBalances / (10**18)

pool_summary = sg.query_df([
    lido_pool_stETH_pool_summary.createdTimestamp,
    lido_pool_stETH_pool_summary.cumulativeProtocolSideRevenueUSD,
    lido_pool_stETH_pool_summary.cumulativeSupplySideRevenueUSD,
    lido_pool_stETH_pool_summary.cumulativeTotalRevenueUSD,
    lido_pool_stETH_pool_summary.inputTokenBalance_decimalConv,
    lido_pool_stETH_pool_summary.outputTokenSupply_decimalConv,
    lido_pool_stETH_pool_summary.totalValueLockedUSD
])
# squeezed_pool_summary = pool_summary.astype('float64').squeeze()
squeezed_pool_summary = pool_summary.squeeze()

def format_decimal(number, decimal_places=4):
    return f"{number:.{decimal_places}f}"

# Assuming your squeezed DataFrame is named 'squeezed_pool_summary'
decimal_pool_summary = squeezed_pool_summary.apply(format_decimal)

# Print the resulting Series
print(decimal_pool_summary)

pools_createdTimestamp                    1608334322.0000
pools_cumulativeProtocolSideRevenueUSD      56169368.6792
pools_cumulativeSupplySideRevenueUSD       498583495.0998
pools_cumulativeTotalRevenueUSD            554752863.7790
pools_inputTokenBalance_decimalConv          5510016.5392
pools_outputTokenSupply_decimalConv          5798052.6872
pools_totalValueLockedUSD                 9113127312.5005
Name: 0, dtype: object
